# 神经网络

对MNIST数据集进行分类

数据集提取为三个部分：mnist.train、mnist.test和mnist.validation，每个部分包含有images和labels。train、test和validation中样本的数量分别为55000、10000和5000。我们可以使用这些数据来训练、测试和验证模型。

## 神经网络设置


定义一个简单的三层全连接网络（输入层、隐藏层和输出层分别为 784、32 和 10 个神经元），隐藏层和输出层的激活函数使用的是 ReLU 函数。该模型训练的样本总数为55000，每次迭代读取的批量为 100。这个简单的全连接网络以交叉熵为损失函数，并使用Adam优化算法进行权重更新。

In [1]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data # Import MNIST
mnist = input_data.read_data_sets("data/MNIST", one_hot=True)

/home/zjw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [2]:
# 载入数据
X_train = mnist.train.images
Y_train = mnist.train.labels
X_test = mnist.test.images
Y_test = mnist.test.labels

# 训练参数设置
hidden_units = 64
learning_rate = 0.01
training_epochs = 20000
display_step = 1000
REGULARIZATION_RATE = 0.0001 #描述网络复杂度的正则化向在损失函数中的系数

In [5]:
w1 = tf.Variable(tf.random_normal([784,hidden_units]))  
w2 = tf.Variable(tf.random_normal([hidden_units,10]))
b1 = tf.Variable(tf.random_normal([hidden_units]))
b2 = tf.Variable(tf.random_normal([10]))


x = tf.placeholder(tf.float32, [None, 784], name='feature-input')  # None表示样本的数量可以是任意，根据batch大小确定维度            
y_ = tf.placeholder(tf.float32, [None, 10], name='label-input')  # 训练数据真实标记

layer1 = tf.nn.relu(tf.matmul(x,w1) + b1)
y = tf.nn.relu(tf.matmul(layer1,w2) + b2)
 
# 定义交叉熵损失函数和训练算法AdamOptimizer
# loss = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0))) #tf.clip_by_value(A, min, max)：输入一个张量A，把A中的每一个元素的值都压缩在min和max之间。小于min的让它等于min，大于max的元素的值等于max。

# loss = -tf.reduce_sum(y_ * tf.log(y))   

# cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
#计算在当前batch中所有样例的交叉熵平均值
loss = tf.reduce_mean(cross_entropy) # cross_entropy_mean

# #计算L2正则化损失
# regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
# #计算网络的正则化损失
# regularization = regularizer(w1) + regularizer(w2)
# #总损失为交叉熵损失和正则化损失之和
# loss = cross_entropy_mean + regularization


train=tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    
    #准备验证数据，在神经网络的训练过程中，会通过验证数据来大致判断停止的条件和评判训练的效果
    validate_data = {x: mnist.validation.images, y_:mnist.validation.labels}
    
    #准备测试数据
    test_data = {x:mnist.test.images, y_:mnist.test.labels}
    
    
    
    for i in range(training_epochs):
        
        batch_xs, batch_ys = mnist.train.next_batch(100) #使用minibatch的训练数据，一个batch的大小为100
        sess.run(train, {x: batch_xs, y_: batch_ys})
        
        if (i + 1) % display_step == 0:
            #计算滑动平均模型在验证数据上的结果，因为MNIST数据集较小，所以可以一次处理所有的验证数据
            validate_acc = sess.run(accuracy, feed_dict=validate_data)
            print("After %d training steps, validation accuracy using average model is %g"
                  %(i+1, validate_acc))
            
    print('Optimize over')
    test_acc = sess.run(accuracy, feed_dict=test_data)
    print("After %d training steps, validation accuracy using average model is %g"
          %(training_epochs, test_acc))

InvalidArgumentError: logits and labels must be same size: logits_size=[100,10] labels_size=[1,100]
	 [[Node: SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape, Reshape_1)]]

Caused by op 'SoftmaxCrossEntropyWithLogits', defined at:
  File "/home/zjw/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/zjw/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py"